## TODO Recording:

- First go to HuggingFace
- Under "Models" search for "T5 small" and show the model card page
- Go to Colab and log in (cloud.user@)
- Import this notebook from local machine
- Go to runtime and show that we are using the TPU
- After running pip install of libraries, please restart the kernel

Installing required huggingface libraries/modules

In [ ]:
pip install transformers datasets evaluate rouge_score accelerate

Checking transformers version

In [ ]:
import transformers

transformers.__version__

'4.31.0'

## TODO Recording

- Go to the HuggingFace site
- Click on the Account on the top right -> Profile -> Settings -> Access Tokens
- Generate a new token and copy it
- Then run the cell below


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

https://huggingface.co/docs/datasets/v1.2.0/loading_datasets.html
Checking all datasets available in huggingface hub

In [ ]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
  print(dataset)

  if (num > 10):
    break

  num += 1

Dataset Name: acronym_identification, Tags: ['task_categories:token-classification', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'source_datasets:original', 'language:en', 'license:mit', 'acronym-identification', 'arxiv:2010.14678', 'region:us']
Dataset Name: ade_corpus_v2, Tags: ['task_categories:text-classification', 'task_categories:token-classification', 'task_ids:coreference-resolution', 'task_ids:fact-checking', 'annotations_creators:expert-generated', 'language_creators:found', 'multilinguality:monolingual', 'size_categories:10K<n<100K', 'size_categories:1K<n<10K', 'size_categories:n<1K', 'source_datasets:original', 'language:en', 'license:unknown', 'region:us']
Dataset Name: adversarial_qa, Tags: ['task_categories:question-answering', 'task_ids:extractive-qa', 'task_ids:open-domain-qa', 'annotations_creators:crowdsourced', 'language_creators:found', 'multilinguality:monolingual', 'size_categori

Loading cnn daily mail dataset

In [ ]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Dataset dictionary can be seen. As the dataset is quite large, we would be taking a subset of training data only.

In [ ]:
cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0", split = "train[:3%]")

cnn_news_summary_ds

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 8613
})

Subset data's dimensions can be checked

In [ ]:
cnn_news_summary_ds.shape

(8613, 3)

dataset description, features, homepage can be obtained
https://huggingface.co/docs/datasets/v1.2.0/exploring.html

In [ ]:
print(cnn_news_summary_ds.description)

CNN/DailyMail non-anonymized summarization dataset.

There are two features:
  - article: text of news article, used as the document to be summarized
  - highlights: joined text of highlights with <s> and </s> around each
    highlight, which is the target summary



In [ ]:
cnn_news_summary_ds.features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [ ]:
cnn_news_summary_ds.homepage

'https://github.com/abisee/cnn-dailymail'

One row of example can be obtained

In [ ]:
cnn_news_summary_ds[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

Dataset is split into train and test sets

In [ ]:
cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed = 42)

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Text cleaning function is defined

In [ ]:
def clean_txt(example):
    for txt in ["article", "highlights"]:
       example[txt]  = example[txt].lower()
       example[txt]  = example[txt].replace("\\", "")
       example[txt]  = example[txt].replace("/", "")
       example[txt]  = example[txt].replace("\n", "")
       example[txt]  = example[txt].replace("``", "")
       example[txt]  = example[txt].replace('"', '')
       example[txt]  = example[txt].replace("--", "")

    return example


Both train and test sets are cleaned using 'map' method

In [ ]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_txt)

cleaned_cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 1723
    })
})

Taking a look at a raw example

In [ ]:
cnn_news_summary_ds["train"]["article"][7]

'(CNN)  -- Four-time defending champion Rafael Nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to Sweden\'s Robin Soderling at the French Open on Sunday. Soderling savors his unlikely triumph against Nadal in the French Open fourth round. Soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at Roland Garros and was a short-priced favorite to win a record fifth straight title. But Soderling never looked back after breezing through the first set of their fourth round match and closed out victory as Nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. The Spaniard tried to look on the positives after his shock defeat. "It\'s not a tragedy, losing here in Paris," he told the official tournament Web site www.rolandgarros.com . "It had to happen one day, and this is an excellent season for me. "Of course it\'s a bit sad, but I have to overcome this as quickly as possible. No one remembers defea

Now observing cleaned up version of the same raw data

In [ ]:
cleaned_cnn_news_summary_ds["train"]["article"][7]

"(cnn)   four-time defending champion rafael nadal slid to an astonishing 6-2 6-7 6-4 7-6 defeat to sweden's robin soderling at the french open on sunday. soderling savors his unlikely triumph against nadal in the french open fourth round. soderling, the 23rd seed, was a rank outsider against the world number one who had never lost a match on the clay at roland garros and was a short-priced favorite to win a record fifth straight title. but soderling never looked back after breezing through the first set of their fourth round match and closed out victory as nadal sent a volley wide after bravely saving a matchpoint in the fourth set tiebreak. the spaniard tried to look on the positives after his shock defeat. it's not a tragedy, losing here in paris, he told the official tournament web site www.rolandgarros.com . it had to happen one day, and this is an excellent season for me. of course it's a bit sad, but i have to overcome this as quickly as possible. no one remembers defeats on the

Similarly,both raw and cleaned up version of highlights(summary) of same example can be seen

In [ ]:
cnn_news_summary_ds["train"]["highlights"][0]

'Maradona staying a clinic in Italy when tax authorities swooped .\nReports: Argentina coach owes millions in taxes from his time playing in Italy .\nMaradona says Serie A club should have paid the taxes .'

In [ ]:
cleaned_cnn_news_summary_ds["train"]["highlights"][0]

'maradona staying a clinic in italy when tax authorities swooped .reports: argentina coach owes millions in taxes from his time playing in italy .maradona says serie a club should have paid the taxes .'

taking a random article for summarising

In [ ]:
EXAMPLE_TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds["test"]["article"][EXAMPLE_TEXT_INDEX]

example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

In [ ]:
MODEL_NAME = "t5-small"

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline("summarization", model = MODEL_NAME)

summary_txt = summarizer(prefix + example_text)

summary_txt

Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors


[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever . lancey, a 10-week-old puppy, was hand-delivered to them earlier this week . he's the most human of any dog we've ever had, the company says ."}]

In [ ]:
summarizer = pipeline("summarization", model = MODEL_NAME, truncation=True)

summary_txt = summarizer(prefix + example_text)

summary_txt

[{'summary_text': 'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'}]

Gold/Reference summary is obtained

In [ ]:
ref_txt = cleaned_cnn_news_summary_ds["test"]["highlights"][EXAMPLE_TEXT_INDEX]

ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

Rouge metric is instantiated

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
reference = "the elephant was found near the river and everyone was glad"

ex_1 = "the big elephant was found near the turbulent river and everyone was ecstatic"

ex_2 = "the big elephant was found near the river and made everyone glad"

Rouge score with extra words in summary in comparison to reference(gold summary)

In [ ]:
results = rouge.compute(predictions = [ex_1], references = [reference])

results

{'rouge1': 0.8333333333333333,
 'rouge2': 0.6363636363636365,
 'rougeL': 0.8333333333333333,
 'rougeLsum': 0.8333333333333333}

Fewer extra words and better the score

In [ ]:
results = rouge.compute(predictions = [ex_2], references = [reference])

results

{'rouge1': 0.8695652173913043,
 'rouge2': 0.5714285714285713,
 'rougeL': 0.8695652173913043,
 'rougeLsum': 0.8695652173913043}

Coming back to our predicted summary

In [ ]:
summary_txt[0]["summary_text"]

'edgar and nina otto paid a california firm $155,000 to clone their beloved labrador retriever . the 10-week-old puppy was hand-delivered to them earlier this week by lou hawthorne .'

Rouge score for that text summary is obtained

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = True)
result

{'rouge1': 0.13157894736842105,
 'rouge2': 0.027027027027027025,
 'rougeL': 0.07894736842105263,
 'rougeLsum': 0.07894736842105263}

All article texts and summaries are defined in a list

In [ ]:
article_texts = cleaned_cnn_news_summary_ds["test"]["article"]

article_summaries = cleaned_cnn_news_summary_ds["test"]["highlights"]

Generating summaries (zero shot) for first 50 rows only as it will takes a very long time in summarizing whole set of examples

In [ ]:
from tqdm import tqdm

candidate_summaries = []

prefix = "summarize: "

for i, text in enumerate(tqdm(article_texts[:50])):

    candidate = summarizer(prefix + text)

    candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [02:54<00:00,  3.48s/it]


Aggregated rouge scores are obtained

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50],
                           use_stemmer = True)
result_agg

{'rouge1': 0.3260855947309701,
 'rouge2': 0.13869246671837798,
 'rougeL': 0.23975283049133256,
 'rougeLsum': 0.24004941836207294}

Here case by case rouge scores are obtained.

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries,
                             references = article_summaries[:50],
                             use_stemmer = True,
                             use_aggregator = False)
result_unagg

{'rouge1': [0.34146341463414637,
  0.38805970149253727,
  0.2608695652173913,
  0.28125,
  0.27848101265822783,
  0.18181818181818182,
  0.3855421686746988,
  0.24615384615384614,
  0.5974025974025975,
  0.3076923076923077,
  0.33766233766233766,
  0.3235294117647059,
  0.17821782178217824,
  0.1917808219178082,
  0.2278481012658228,
  0.3582089552238805,
  0.2702702702702703,
  0.3235294117647059,
  0.2682926829268293,
  0.32432432432432434,
  0.35443037974683544,
  0.28205128205128205,
  0.40540540540540543,
  0.10666666666666667,
  0.3116883116883117,
  0.36363636363636365,
  0.32558139534883723,
  0.5757575757575758,
  0.42857142857142855,
  0.24444444444444444,
  0.271604938271605,
  0.37288135593220334,
  0.3,
  0.13157894736842105,
  0.3820224719101123,
  0.44,
  0.4507042253521127,
  0.5217391304347826,
  0.647887323943662,
  0.2105263157894737,
  0.1038961038961039,
  0.2153846153846154,
  0.1590909090909091,
  0.3711340206185567,
  0.3548387096774194,
  0.2571428571428571,
  

Minimum and maximum Rouge score indices are obtained to check best and worst summaries

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg["rouge2"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(38, 12)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,firefighter jimmy senk went inside a burning a...,horia cretan became a hero for carrying boy fr...
1,nigeria's princess nikky onyeri has dedicated ...,health advocate from nigeria campaigns for bet...
2,troy davis has always maintained his innocence...,supreme court's latest ruling means troy davis...
3,"more than 11,500 public servants have been sus...",international transparency poll: businesses in...
4,human rights watch says the displaced sri lank...,"group: more than 280,000 displaced tamil civil..."


For the best rouge score, we can observe the predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(38, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(38, "Reference_summaries"))

Actual Summary
the statue of kate moss will be displayed in the british museum in october . it is the largest gold statue built since ancient egypt . the statue will be part of a group of major sculptures .
Reference Summary
statue of kate moss billed as largest gold statue built since ancient egypt .called siren, the statue will be on display at the british museum .moss statue will be displayed in gallery housing ancient greek sculpture .


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(12, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(12, "Reference_summaries"))

Actual Summary
mobile computer chips have gotten so powerful and affordable, and it doesn't make much sense to ship a gadget with dedicated, stagnant functionality . developers have created 100,000 apps for the store at no cost to apple, which takes a cut of revenues .
Reference Summary
companies are opening app stores and bringing applications to tvs, netbooks and even cars .intel launched an app store for netbooks at the consumer electronics show this week .samsung will launch an app store later this year for hdtvs and blu-ray players .ford motor co. will make select apps available for new vehicles later in 2010 .


Loading  a T5 tokenizer to process text and summary

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Tokenizer is applied and token ids can be seen

In [ ]:
tokenizer(["four-time defending champion"])

{'input_ids': [[662, 18, 715, 3, 20309, 6336, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
tokens = tokenizer.tokenize("four-time defending champion")

tokens

['▁four', '-', 'time', '▁', 'defending', '▁champion']

In [ ]:
ids = tokenizer.convert_tokens_to_ids(tokens)

ids

[662, 18, 715, 3, 20309, 6336]

Loading T5 Model with AutoModelForSeq2SeqLM

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

The preprocessing function you want to create needs to:

Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
Use the keyword text_target argument when tokenizing labels.
Truncate sequences to be no longer than the maximum length set by the max_length parameter.

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples["highlights"], max_length = 128, truncation = True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

Applying the preprocessing function over the entire dataset by using 'map' method. We can speed up the map function by setting batched=True to process multiple elements of the dataset at once

In [ ]:
tokenized_cnn_news_summary_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched = True)

tokenized_cnn_news_summary_ds

Map:   0%|          | 0/6890 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6890
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1723
    })
})

Now create a batch of examples using DataCollatorForSeq2Seq. It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["generated_length"] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}


Defining our training hyperparameters in Seq2SeqTrainingArguments. The only required parameter is output_dir which specifies where to save your model. We will push this model to the Hub by setting push_to_hub=True (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the Trainer will evaluate the ROUGE metric and save the training checkpoint.
Passing the training arguments to Seq2SeqTrainer along with the model, dataset, tokenizer, data collator, and compute_metrics function.
Calling train() to finetune your model.

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "cnn_news_summary_model_trained_on_reduced_data",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 3,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds["train"],
    eval_dataset = tokenized_cnn_news_summary_ds["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

/content/cnn_news_summary_model_trained_on_reduced_data is already a clone of https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,1.600637,0.218100,0.094400,0.183700,0.183800,19.000000
2,1.808300,1.592310,0.218700,0.095200,0.184200,0.184500,19.000000
3,1.800400,1.590927,0.218400,0.095100,0.184100,0.184300,19.000000


TrainOutput(global_step=1293, training_loss=1.8007458939814143, metrics={'train_runtime': 1435.8847, 'train_samples_per_second': 14.395, 'train_steps_per_second': 0.9, 'total_flos': 5595030072852480.0, 'train_loss': 1.8007458939814143, 'epoch': 3.0})

## TODO Recording:

- Immediately after training completes click on the link to the model on HuggingFace e.g. https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data
- Click on Model Card - should be nothing there
- Click on Files and Versions - should see the trained model files and some commits there
- Click on Training Metrics - that will take you to TensorBoard - show the graphs there (click on all tabs)

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/231M [00:00<?, ?B/s]

Upload file runs/Aug06_13-13-19_c43c2ceb9006/events.out.tfevents.1691327602.c43c2ceb9006.50464.1:   0%|       …

To https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data
   6deb92d..6f0f6bb  main -> main

   6deb92d..6f0f6bb  main -> main

To https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data
   6f0f6bb..0a042df  main -> main

   6f0f6bb..0a042df  main -> main



'https://huggingface.co/loony-user/cnn_news_summary_model_trained_on_reduced_data/commit/6f0f6bb18d7b1039b5b343ca9977b2a3c2f5e231'

## TODO Recording:

- Now go to the model on HuggingFace
- Click on "Model Card" show that it has been generated
- Click on "Files and Versions" and show

In [ ]:
example_text

"(cnn)  every dog has his day, but sir lancelot  or at least his carbon copy  has a second one. edgar and nina otto show off 10-week-old lancey, a clone of the most human of any dog we've ever had. a boca raton, florida, couple paid a california firm $155,000 to clone their beloved labrador retriever, who died from cancer a year ago. the clone, a 10-week-old puppy dubbed lancey, was hand-delivered to them earlier this week by lou hawthorne, chairman of bioarts international, a biotechnology company. one minute with lancey and you know he's special. he's both extremely aware and very sweet, hawthorne said in a bioarts statement. edgar and nina otto said they began thinking about cloning sir lancelot about five years ago. i said 'well, you know, it wouldn't hurt to have his dna frozen,' and that's what we did, nina otto told cnn affiliate wpbf. the ottos were one of five families to bid and win a bioarts auction for a chance to clone their family dog, according to a bioarts statement. la

The simplest way to try out your finetuned model for inference is to use it in a pipeline(). Instantiate a pipeline for summarization with your model, and pass your text to it:

In [ ]:
from transformers import pipeline

prefix = "summarize: "

summarizer = pipeline(
    "summarization",
    model = "loony-user/cnn_news_summary_model_trained_on_reduced_data",
    truncation=True
)
summary_txt = summarizer(prefix + example_text)

summary_txt

[{'summary_text': "edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever .the 10-week-old puppy, dubbed lancey, was hand-delivered to them earlier this week .otos were one of five families to bid and win a bioarts auction for a chance .sir lancelot's dna sample was sent to the sooam biotech research foundation in"}]

Alternative way of summarising text(manual method)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")

inputs = tokenizer(prefix + example_text, return_tensors = "pt", truncation=True).input_ids

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("loony-user/cnn_news_summary_model_trained_on_reduced_data")

outputs = model.generate(inputs, max_new_tokens = 100, do_sample = False)

tokenizer.decode(outputs[0], skip_special_tokens = True)

'edgar and nina otto paid california firm $155,000 to clone their beloved labrador retriever.the 10-week-old puppy dubbed lancey was hand-delivered to them earlier this week.ottos are the first of six current clients to receive their clone.'

In [ ]:
ref_txt

"couple won auction to clone family dog, biotech company says .lancey is world's first commercially cloned dog, company says .dna of deceased dog sent to s. korea, and cloned puppy born november 18 .humane society says it's against commercial cloning of animals ."

In [ ]:
result = rouge.compute(predictions = [summary_txt[0]["summary_text"]],
                       references = [ref_txt], use_stemmer = False)
result

{'rouge1': 0.23076923076923075,
 'rouge2': 0.0392156862745098,
 'rougeL': 0.13461538461538464,
 'rougeLsum': 0.13461538461538464}

In [ ]:
candidate_summaries = []

for i, text in enumerate(tqdm(article_texts[:50])):

  candidate = summarizer(prefix + text)

  candidate_summaries.append(candidate[0]["summary_text"])

100%|██████████| 50/50 [04:02<00:00,  4.86s/it]


Looks like zero shot untuned model is having slighly better performance than our fine tuned model.

In [ ]:
result_agg = rouge.compute(predictions = candidate_summaries,
                           references = article_summaries[:50], use_stemmer = False)
result_agg

{'rouge1': 0.33933829483177846,
 'rouge2': 0.15359698297170693,
 'rougeL': 0.25293919302055723,
 'rougeLsum': 0.2519742018631588}

In [ ]:
result_unagg = rouge.compute(predictions = candidate_summaries, references = article_summaries[:50],
                             use_stemmer = True, use_aggregator = False)

In [ ]:
result_unagg_rsum = np.array(result_unagg["rougeLsum"])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(27, 23)

Actual vs Predicted summaries dataframe is obtained

In [ ]:
act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:50])),
                                        columns = ["Predicted_Summaries", "Reference_summaries"])
act_vs_pred_summaries_df.head()

,Predicted_Summaries,Reference_summaries
0,bronx firefighter jimmy senk pulls 4-year-old ...,horia cretan became a hero for carrying boy fr...
1,nigeria's princess nikky onyeri speaks out to ...,health advocate from nigeria campaigns for bet...
2,troy davis has always maintained his innocence...,supreme court's latest ruling means troy davis...
3,"more than 11,500 public servants suspended or ...",international transparency poll: businesses in...
4,sri lankan government should release more than...,"group: more than 280,000 displaced tamil civil..."


Taking a look at the highest and lowest rouge score case's predicted and reference summaries

In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(27, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(27, "Reference_summaries"))

Actual Summary
cnn's mainsail investigates what makes a classic yacht at one of the world's great yachting regattas .shirley robertson hitches a ride on board some of the most iconic yachts both ancient and modern .alghi team boss ernesto bertarelli drives the new alinghi 5 the defender of the america's cup .
Reference Summary
what makes a 'classic' yacht? mainsail takes up the challenge to find out .shirley robertson hitches a ride on some of the world's most iconic yachts .she drives the new alinghi 5, while chatting to alinghi team boss bertarelli .


In [ ]:
print("Actual Summary")
print(act_vs_pred_summaries_df._get_value(23, "Predicted_Summaries"))

print("Reference Summary")
print(act_vs_pred_summaries_df._get_value(23, "Reference_summaries"))

Actual Summary
chandra levy's mother flew to atlanta just to talk to them .they mailed their findings to the police chief in washington, d.c. they never heard back, but on saturday, the text and phone messages began to fly .a week ago, the washington police chief called his parents .
Reference Summary
group can't divulge whom it named as a suspect in chandra levy case .arrest warrant completely validates 15 months of work, professor says .professor recalls levy's mother being grateful that students kept spotlight on case .about 80 students at bauder college are members of crime club .
